In [1]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [2]:
import numpy as np

In [3]:
actions = np.array(['hello', 'thanks', 'bye', 'good', 'congrats'])

In [4]:
label_map = {label:num for num,label in enumerate(actions)}

In [5]:
label_map

{'hello': 0, 'thanks': 1, 'bye': 2, 'good': 3, 'congrats': 4}

In [7]:
import os

In [8]:
DATA_PATH = os.path.join('MP_Data')
actions = np.array(['hello', 'thanks', 'bye', 'good', 'congrats'])
no_sequences = 50        
sequence_length = 20

## PreProcess

In [10]:
sequences, labels = [],[]
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH,action,str(sequence),"{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [11]:
sequences

[[array([ 0.50960326,  0.35896713, -0.47463471, ...,  0.46930823,
          0.27105767, -0.0111786 ]),
  array([ 0.50996375,  0.35930899, -0.46962413, ...,  0.47003803,
          0.27239132, -0.01250922]),
  array([ 0.51003104,  0.35969013, -0.46995029, ...,  0.4711982 ,
          0.27259505, -0.01234089]),
  array([ 0.51036012,  0.35996345, -0.47365069, ...,  0.47064352,
          0.27277651, -0.01160482]),
  array([ 0.51080102,  0.36001518, -0.47675186, ...,  0.47064632,
          0.2722621 , -0.01225885]),
  array([ 0.5133751 ,  0.36227366, -0.48237523, ...,  0.4705171 ,
          0.27379188, -0.01252029]),
  array([ 0.51535547,  0.36236328, -0.49322972, ...,  0.47024947,
          0.2733666 , -0.01156798]),
  array([ 0.51631868,  0.36276454, -0.49571729, ...,  0.46979845,
          0.27344143, -0.01244814]),
  array([ 0.51952302,  0.36324522, -0.49790746, ...,  0.47007152,
          0.27562901, -0.01188941]),
  array([ 0.51952624,  0.36026353, -0.4978953 , ...,  0.46924806,
       

In [12]:
np.array(sequences).shape

(250, 20, 1662)

In [13]:
X = np.array(sequences)

In [14]:
X.shape

(250, 20, 1662)

In [15]:
y = to_categorical(labels).astype('int')

In [16]:
y

array([[1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1]])

In [17]:
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.05)

In [18]:
y_test.shape

(13, 5)

## Train Model

In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense
from tensorflow.keras.callbacks import TensorBoard

In [21]:
#use to monitor the model while training
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [61]:
model = Sequential()
model.add(LSTM(64,return_sequences=True,activation='relu',input_shape=(20,1662)))
model.add(LSTM(128,return_sequences=True,activation='relu'))
model.add(LSTM(64,return_sequences=False,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(32,activation='relu'))
model.add(Dense(actions.shape[0],activation='softmax'))

In [62]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['categorical_accuracy'])

In [63]:
model.fit(x_train,y_train,epochs=100,callbacks=[tb_callback])

Epoch 1/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 46ms/step - categorical_accuracy: 0.1749 - loss: 1.7232
Epoch 2/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - categorical_accuracy: 0.1677 - loss: 1.6650
Epoch 3/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - categorical_accuracy: 0.2737 - loss: 1.5511
Epoch 4/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - categorical_accuracy: 0.3939 - loss: 1.4222
Epoch 5/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - categorical_accuracy: 0.2765 - loss: 1.4192
Epoch 6/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - categorical_accuracy: 0.2521 - loss: 1.5136
Epoch 7/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - categorical_accuracy: 0.4621 - loss: 1.2272
Epoch 8/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - categorical_accuracy: 0.6253 - loss: 0.8378
Epoch 9/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - categorical_accuracy: 0.5135 - loss: 1.2173
Epoch 10/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - categorical_accuracy: 0.3243 - loss: 1.3306
Epoch 11/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0

In [55]:
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_9 (LSTM)                        │ (None, 20, 64)              │         442,112 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_10 (LSTM)                       │ (None, 20, 128)             │          98,816 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_11 (LSTM)                       │ (None, 64)                  │          49,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 64)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ (None, 5)                   │             165 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,790,225 (6.83 MB)

 Trainable params: 596,741 (2.28 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,193,484 (4.55 MB)

## Prediction

In [56]:
res = model.predict(x_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 309ms/step


In [57]:
actions[np.argmax(res[0])]

'bye'

In [58]:
actions[np.argmax(y_test[0])]

'bye'

## Saving the Model

In [59]:
model.save("models/Dynamic.h5")

In [60]:
del model

In [37]:
model.load_weights("models/Dynamic.h5")

## Evaluation

In [38]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [46]:
yhat = model.predict(x_train)

8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step 


In [47]:
ytrue = np.argmax(y_train,axis=1).tolist()
yhat = np.argmax(yhat,axis=1).tolist()

In [48]:
ytrue

[1,
 3,
 0,
 1,
 3,
 2,
 1,
 3,
 4,
 3,
 0,
 2,
 2,
 3,
 0,
 3,
 3,
 0,
 3,
 0,
 4,
 3,
 3,
 1,
 4,
 4,
 2,
 3,
 2,
 1,
 2,
 3,
 2,
 2,
 4,
 1,
 2,
 1,
 4,
 2,
 4,
 3,
 1,
 0,
 4,
 0,
 1,
 1,
 1,
 3,
 2,
 3,
 2,
 4,
 4,
 4,
 0,
 3,
 2,
 2,
 4,
 2,
 0,
 4,
 0,
 0,
 2,
 3,
 3,
 4,
 3,
 3,
 0,
 3,
 0,
 1,
 1,
 4,
 3,
 3,
 2,
 4,
 0,
 1,
 2,
 3,
 3,
 2,
 1,
 1,
 3,
 1,
 2,
 2,
 1,
 0,
 0,
 2,
 1,
 3,
 4,
 3,
 2,
 3,
 0,
 1,
 3,
 2,
 2,
 1,
 2,
 3,
 2,
 1,
 3,
 0,
 3,
 4,
 3,
 4,
 0,
 4,
 3,
 1,
 3,
 2,
 1,
 1,
 0,
 4,
 0,
 0,
 4,
 4,
 1,
 1,
 0,
 0,
 1,
 4,
 4,
 4,
 2,
 4,
 4,
 3,
 1,
 4,
 0,
 0,
 3,
 0,
 0,
 1,
 4,
 1,
 0,
 0,
 4,
 3,
 4,
 4,
 2,
 0,
 2,
 2,
 4,
 0,
 2,
 4,
 1,
 2,
 1,
 1,
 4,
 2,
 1,
 1,
 0,
 1,
 3,
 0,
 2,
 2,
 0,
 2,
 3,
 3,
 3,
 0,
 2,
 2,
 2,
 0,
 4,
 4,
 3,
 2,
 0,
 4,
 4,
 2,
 4,
 1,
 1,
 3,
 0,
 1,
 4,
 0,
 3,
 0,
 2,
 1,
 1,
 4,
 0,
 4,
 0,
 0,
 0,
 0,
 2,
 1,
 4,
 4,
 3,
 3,
 3,
 2,
 1,
 4,
 4,
 3,
 1,
 4,
 1]

In [49]:
yhat

[1,
 3,
 0,
 1,
 3,
 2,
 1,
 3,
 0,
 3,
 0,
 0,
 2,
 3,
 0,
 3,
 3,
 0,
 3,
 0,
 1,
 3,
 3,
 1,
 4,
 4,
 2,
 3,
 2,
 1,
 2,
 3,
 2,
 2,
 4,
 1,
 2,
 1,
 4,
 2,
 4,
 3,
 1,
 0,
 4,
 0,
 1,
 1,
 1,
 3,
 2,
 3,
 2,
 4,
 4,
 4,
 0,
 3,
 2,
 2,
 4,
 2,
 0,
 4,
 0,
 0,
 2,
 3,
 3,
 4,
 3,
 3,
 0,
 3,
 0,
 1,
 1,
 4,
 3,
 3,
 2,
 4,
 0,
 1,
 2,
 2,
 3,
 2,
 1,
 1,
 2,
 0,
 2,
 2,
 1,
 0,
 0,
 2,
 1,
 3,
 4,
 3,
 2,
 3,
 0,
 1,
 3,
 2,
 2,
 1,
 2,
 3,
 2,
 1,
 3,
 0,
 3,
 4,
 3,
 4,
 0,
 4,
 3,
 1,
 3,
 2,
 4,
 1,
 0,
 4,
 0,
 0,
 4,
 4,
 1,
 1,
 0,
 0,
 1,
 4,
 4,
 4,
 2,
 4,
 4,
 2,
 4,
 4,
 0,
 0,
 3,
 0,
 0,
 1,
 4,
 1,
 0,
 0,
 4,
 2,
 4,
 4,
 2,
 0,
 2,
 2,
 4,
 0,
 2,
 1,
 1,
 2,
 1,
 1,
 4,
 2,
 1,
 1,
 0,
 1,
 3,
 0,
 2,
 2,
 0,
 2,
 3,
 3,
 3,
 0,
 2,
 2,
 2,
 0,
 4,
 4,
 3,
 2,
 0,
 4,
 4,
 2,
 4,
 1,
 1,
 3,
 0,
 1,
 4,
 0,
 3,
 0,
 2,
 1,
 1,
 4,
 0,
 4,
 0,
 0,
 0,
 0,
 2,
 1,
 4,
 4,
 3,
 3,
 3,
 2,
 1,
 4,
 4,
 3,
 1,
 4,
 1]

In [50]:
multilabel_confusion_matrix(ytrue,yhat)

array([[[188,   3],
        [  0,  46]],

       [[189,   2],
        [  3,  43]],

       [[187,   4],
        [  1,  45]],

       [[187,   0],
        [  4,  46]],

       [[186,   2],
        [  3,  46]]], dtype=int64)

In [51]:
accuracy_score(ytrue,yhat)

0.9535864978902954